In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'2hgx9x0t'

In [3]:
wandb.init(id='2hgx9x0t', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 1min 5s, sys: 1min 30s, total: 2min 36s
Wall time: 2min 39s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 5.96 µs


In [9]:
"""for n, p in m.named_parameters():
    if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False
        
    print(n, p.requires_grad)"""

"for n, p in m.named_parameters():\n    if 'fcs_2' in n:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False\n        \n    print(n, p.requires_grad)"

In [10]:
set_lr(1e-4)

In [11]:
model_stem = "9.24"
START_E = 0

In [12]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = get_lr() #6e-4 if e < 40 else 3e-4 if e < 80 else 1e-4

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 0

{'trn_control_loss': 0.06080043, 'trn_headings_loss': 0.00173762, 'trn_curvatures_loss': 0.00366221, 'consistency losses/trn_steer_cost': 0.00597878, 'consistency losses/trn_te_loss': 23.11392704, 'aux losses/trn_pitch_loss': 1.42541504, 'aux losses/trn_yaw_loss': 1.19146729, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9609375, 'consistency losses/trn_torque_loss': 4401.83154678, 'consistency losses/trn_torque_delta_loss': 21.13203117, 'logistical/obs_consumed_per_second': 202.56692913, 'logistical/obs_generated_per_second': 166.51181102, 'logistical/data_consumption_ratio': 1.21560876, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.25979424, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 197.453125, 'logistical/obs_generated_per_second': 167.4921875, 'logistical/data_consumption_ratio': 1.17955438, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.04341304, 'trn_headings_loss': 0.00144584, 'trn_curvatures_loss': 0.003

{'logistical/obs_consumed_per_second': 200.0859375, 'logistical/obs_generated_per_second': 168.8828125, 'logistical/data_consumption_ratio': 1.18429313, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01715052, 'trn_headings_loss': 0.00066156, 'trn_curvatures_loss': 0.00219923, 'consistency losses/trn_steer_cost': 0.00585024, 'consistency losses/trn_te_loss': 22.61455758, 'aux losses/trn_pitch_loss': 1.36239624, 'aux losses/trn_yaw_loss': 0.53441429, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8984375, 'consistency losses/trn_torque_loss': 4221.47083282, 'consistency losses/trn_torque_delta_loss': 24.36987418, 'logistical/max_param': 16.25907135, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 195.875, 'logistical/obs_generated_per_second': 168.4765625, 'logistical/data_consumption_ratio': 1.1627258, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01302209, 'trn_headings_loss': 0.00059306, 'trn_curvatures_loss': 0.00191779, 'consist

{'logistical/obs_consumed_per_second': 192.5390625, 'logistical/obs_generated_per_second': 167.7578125, 'logistical/data_consumption_ratio': 1.14912772, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01440489, 'trn_headings_loss': 0.00069257, 'trn_curvatures_loss': 0.00226782, 'consistency losses/trn_steer_cost': 0.00550133, 'consistency losses/trn_te_loss': 22.74579872, 'aux losses/trn_pitch_loss': 1.18257904, 'aux losses/trn_yaw_loss': 0.47529125, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.96875, 'consistency losses/trn_torque_loss': 4261.98021698, 'consistency losses/trn_torque_delta_loss': 20.73700406, 'logistical/max_param': 16.26711845, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 198.6171875, 'logistical/obs_generated_per_second': 175.6640625, 'logistical/data_consumption_ratio': 1.13118966, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01388124, 'trn_headings_loss': 0.00062123, 'trn_curvatures_loss': 0.00195919, 'cons

{'logistical/obs_consumed_per_second': 199.6875, 'logistical/obs_generated_per_second': 174.796875, 'logistical/data_consumption_ratio': 1.14443215, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01775157, 'trn_headings_loss': 0.0008138, 'trn_curvatures_loss': 0.00299879, 'consistency losses/trn_steer_cost': 0.00658111, 'consistency losses/trn_te_loss': 22.3666005, 'aux losses/trn_pitch_loss': 1.44376373, 'aux losses/trn_yaw_loss': 0.57215118, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.921875, 'consistency losses/trn_torque_loss': 3977.68870544, 'consistency losses/trn_torque_delta_loss': 19.55372902, 'logistical/max_param': 16.26249695, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 194.609375, 'logistical/obs_generated_per_second': 174.9296875, 'logistical/data_consumption_ratio': 1.11220172, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01515293, 'trn_headings_loss': 0.0007245, 'trn_curvatures_loss': 0.0025823, 'consistency 

{'logistical/obs_consumed_per_second': 198.75, 'logistical/obs_generated_per_second': 162.2421875, 'logistical/data_consumption_ratio': 1.22551167, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01147842, 'trn_headings_loss': 0.00055032, 'trn_curvatures_loss': 0.00140259, 'consistency losses/trn_steer_cost': 0.00392501, 'consistency losses/trn_te_loss': 24.03541279, 'aux losses/trn_pitch_loss': 1.04121399, 'aux losses/trn_yaw_loss': 0.43086433, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 4779.43505287, 'consistency losses/trn_torque_delta_loss': 24.99910263, 'logistical/max_param': 16.26066399, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 194.2265625, 'logistical/obs_generated_per_second': 168.3359375, 'logistical/data_consumption_ratio': 1.15363569, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01209968, 'trn_headings_loss': 0.0005095, 'trn_curvatures_loss': 0.0015405, 'consisten

...

{'logistical/obs_consumed_per_second': 196.3046875, 'logistical/obs_generated_per_second': 159.171875, 'logistical/data_consumption_ratio': 1.23412247, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00841874, 'trn_headings_loss': 0.00044459, 'trn_curvatures_loss': 0.00172113, 'consistency losses/trn_steer_cost': 0.00379878, 'consistency losses/trn_te_loss': 24.83695544, 'aux losses/trn_pitch_loss': 1.27909088, 'aux losses/trn_yaw_loss': 0.24960852, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9453125, 'consistency losses/trn_torque_loss': 4157.05684662, 'consistency losses/trn_torque_delta_loss': 22.97642823, 'logistical/max_param': 16.27622986, 'logistical/lr': 5e-05}
{'logistical/obs_consumed_per_second': 194.90625, 'logistical/obs_generated_per_second': 159.7421875, 'logistical/data_consumption_ratio': 1.22211758, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00882, 'trn_headings_loss': 0.00041151, 'trn_curvatures_loss': 0.00212326, 'consisten

{'logistical/obs_consumed_per_second': 128.0, 'logistical/obs_generated_per_second': 161.0, 'logistical/data_consumption_ratio': 0.79448879, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 60

{'trn_control_loss': 0.01053828, 'trn_headings_loss': 0.00061475, 'trn_curvatures_loss': 0.00194881, 'consistency losses/trn_steer_cost': 0.00537124, 'consistency losses/trn_te_loss': 24.03872485, 'aux losses/trn_pitch_loss': 1.40734863, 'aux losses/trn_yaw_loss': 0.25884676, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9609375, 'consistency losses/trn_torque_loss': 4353.96559715, 'consistency losses/trn_torque_delta_loss': 26.51068931, 'logistical/obs_consumed_per_second': 195.86614173, 'logistical/obs_generated_per_second': 165.16535433, 'logistical/data_consumption_ratio': 1.18614112, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.27625275, 'logistical/lr': 5e-05}
{'logistical/obs_consumed_per_second': 194.78125, 'logistical/obs_generated_per_second': 165.

{'logistical/obs_consumed_per_second': 193.0, 'logistical/obs_generated_per_second': 166.5625, 'logistical/data_consumption_ratio': 1.158799, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01144275, 'trn_headings_loss': 0.00043724, 'trn_curvatures_loss': 0.00168717, 'consistency losses/trn_steer_cost': 0.00557691, 'consistency losses/trn_te_loss': 22.2852077, 'aux losses/trn_pitch_loss': 1.30934525, 'aux losses/trn_yaw_loss': 0.31774473, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9453125, 'consistency losses/trn_torque_loss': 4181.81938744, 'consistency losses/trn_torque_delta_loss': 18.2238221, 'logistical/max_param': 16.27805901, 'logistical/lr': 5e-05}
{'logistical/obs_consumed_per_second': 190.3984375, 'logistical/obs_generated_per_second': 163.7734375, 'logistical/data_consumption_ratio': 1.16412997, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01040447, 'trn_headings_loss': 0.00071528, 'trn_curvatures_loss': 0.00186988, 'consistency loss

{'logistical/obs_consumed_per_second': 194.2578125, 'logistical/obs_generated_per_second': 167.7265625, 'logistical/data_consumption_ratio': 1.15848957, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01213107, 'trn_headings_loss': 0.00064345, 'trn_curvatures_loss': 0.00247497, 'consistency losses/trn_steer_cost': 0.00565098, 'consistency losses/trn_te_loss': 22.26608475, 'aux losses/trn_pitch_loss': 1.07093048, 'aux losses/trn_yaw_loss': 0.2977457, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.96875, 'consistency losses/trn_torque_loss': 4438.19441223, 'consistency losses/trn_torque_delta_loss': 19.02112243, 'logistical/max_param': 16.27257919, 'logistical/lr': 5e-05}
{'logistical/obs_consumed_per_second': 196.4453125, 'logistical/obs_generated_per_second': 165.3359375, 'logistical/data_consumption_ratio': 1.18834757, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01179847, 'trn_headings_loss': 0.00043835, 'trn_curvatures_loss': 0.00164953, 'consis

KeyboardInterrupt: 

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e63.torch")

In [14]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
